<a href="https://colab.research.google.com/github/OptimumPrimal001/OOU_CSC309_Artificial_Intelligence/blob/main/notebooks/CSC309_Week03_Search_BFS_DFS_Student_Centred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC309 – Artificial Intelligence  
**Week 3 Lab:** Search & Problem Solving — BFS/DFS on Classic Problems

**Instructor:** Dr Sakinat Folorunso

**Title:** Associate Professor of AI Systems and FAIR Data **Department:** Computer Sciences, Olabisi Onabanjo University, Ago-Iwoye, Ogun State, Nigeria

**Course Code:** CSC 309

**Mode:** Student‑centred, hands‑on in Google Colab

> Every code cell is commented line‑by‑line so you can follow the logic precisely.

## How to use this notebook
1. Start with the **Group Log** and **Do Now**.  
2. Run the **Setup** cell once.  
3. Work through **Tasks**. Edit only cells marked **`# TODO(Student)`**.  
4. Use **Quick Checks** to test your understanding.  
5. Finish with the **Reflection**. If you finish early, try the **Extensions**.

In [ ]:
#@title 🧑🏽‍🤝‍🧑🏾 Group Log (fill before you start)
# The '#@param' annotations create form fields in Colab for easy input.

group_members = "AKINWANDE JOSHUA, OLUFEMI DANIEL, ADEGOKE ANUOOLUWAPO, BALOGUN EMMANUEL, AWOTEDU BOLUWATIFE,ADEBOYEJO ISAIAH, ADELEYE ARAFAT, ODUKOYA ADEBIMPE, ADEOSUN OLUWADEMILADE, NOIRU SAMUEL, ONABOWU ABDULLAH"  #@param {type:"string"}  # Names of teammates
roles_notes = "Driver/Navigator, decisions, questions"  #@param {type:"string"}  # Short working notes

print("👥 Group:", group_members)        # Echo the group list for confirmation
print("📝 Notes:", roles_notes)          # Echo the notes so they're preserved in output

👥 Group: AKINWANDE JOSHUA, OLUFEMI DANIEL, ADEGOKE ANUOOLUWAPO, BALOGUN EMMANUEL, AWOTEDU BOLUWATIFE,ADEBOYEJO ISAIAH, ADELEYE ARAFAT, ODUKOYA ADEBIMPE, ADEOSUN OLUWADEMILADE, NOIRU SAMUEL, ONABOWU ABDULLAH
📝 Notes: Driver/Navigator, decisions, questions


### Learning Objectives
- Formulate problems as **state spaces**.  
- Implement **BFS** and **DFS** with path reconstruction.  
- Discuss completeness/optimality and **combinatorial explosion**.

In [ ]:
#@title 🔧 Setup (no extra installs)
# BFS/DFS rely only on Python's standard library.

from collections import deque     # 'deque' gives an efficient queue for BFS
print("✅ Setup complete for Week 3.")

✅ Setup complete for Week 3.


In [ ]:
#@title 🧭 BFS and DFS (fully commented)

def bfs(start, is_goal, neighbors):
    """Breadth‑First Search returning the path from start to goal or None."""
    frontier = deque([start])        # Queue of discovered states to expand
    parent = {start: None}           # Dictionary mapping state -> predecessor for path

    while frontier:                  # Continue until queue is empty
        s = frontier.popleft()       # Pop the oldest state (FIFO)
        if is_goal(s):               # If this state is a goal
            path = []                # We'll reconstruct the path by walking parents
            while s is not None:     # Walk backward until the start (None parent)
                path.append(s)       # Add current state to the path
                s = parent[s]        # Move to predecessor
            return list(reversed(path))  # Reverse to get start..goal order

        for n in neighbors(s):       # For each neighbor of the current state
            if n not in parent:      # Skip already‑seen states
                parent[n] = s        # Record how we reached 'n'
                frontier.append(n)   # Push 'n' to the back of the queue
    return None                      # No path found

def dfs(start, is_goal, neighbors, limit=10000):
    """Depth‑First Search with a node expansion limit; returns a path or None."""
    stack = [start]                  # Use a Python list as a LIFO stack
    parent = {start: None}           # Predecessor map for path reconstruction
    steps = 0                        # Counter to avoid infinite exploration

    while stack and steps < limit:   # Stop if stack empty or limit reached
        s = stack.pop()              # Pop the most recent state (LIFO)
        steps += 1                   # Count this expansion
        if is_goal(s):               # If we reached a goal
            path = []                # Reconstruct path exactly like BFS
            while s is not None:
                path.append(s)
                s = parent[s]
            return list(reversed(path))
        for n in neighbors(s):       # Consider neighbors
            if n not in parent:      # Only take unseen states
                parent[n] = s        # Remember predecessor
                stack.append(n)      # Push neighbor to stack
    return None                      # No path found (or limit exceeded)

In [ ]:
#@title ⛵ Missionaries & Cannibals (neighbors function, fully commented)

def mc_neighbors(state):
    """Return legal successor states for (M_left, C_left, boat_side)."""
    M, C, side = state                  # Unpack state; side: 0=left, 1=right
    moves = [(1,0),(2,0),(0,1),(0,2),(1,1)]  # Boat can carry these combinations
    result = []                          # Accumulate legal next states here
    for m, c in moves:                   # Try each boat payload
        if side == 0:                    # If boat is on left bank
            M2, C2, side2 = M - m, C - c, 1   # Move people from left to right
        else:                            # If boat is on right bank
            M2, C2, side2 = M + m, C + c, 0   # Move people from right to left
        if 0 <= M2 <= 3 and 0 <= C2 <= 3:      # Keep counts within [0,3]
            # Safety on left: missionaries not outnumbered (unless zero missionaries)
            safe_left = (M2 == 0 or M2 >= C2)
            # Safety on right (computed from complements 3-M2, 3-C2)
            safe_right = ((3 - M2) == 0) or ((3 - M2) >= (3 - C2))
            if safe_left and safe_right:       # Only accept safe states
                result.append((M2, C2, side2)) # Add the legal successor
    return result                              # Return list of legal neighbors

# Quick demo: BFS path length from start to goal
start = (3, 3, 0)                     # All on the left bank, boat on left
goal  = (0, 0, 1)                     # All on the right bank, boat on right
path_bfs = bfs(start, lambda s: s == goal, mc_neighbors)   # Run BFS
print("BFS path length:", len(path_bfs) if path_bfs else None)  # Show result length

BFS path length: 12


### Task — Uniform‑Cost Search (UCS)
Extend BFS into **UCS** using a priority queue; assign unit cost to each boat move.  
Then compare DFS/BFS/UCS by **number of states expanded**.

Uniform-Cost Search (UCS) + Comparison

In [2]:


import heapq

def ucs(start, is_goal, neighbors):
    """
    Uniform-Cost Search (UCS) using a priority queue (min-heap).
    Every action has cost = 1.
    Returns (path, expansions)
    """
    frontier = [(0, start)]      # (cost, state)
    parent = {start: None}       # for reconstructing path
    cost_so_far = {start: 0}     # g(n)
    expansions = 0               # to count nodes expanded

    while frontier:
        cost, state = heapq.heappop(frontier)
        expansions += 1          # we expanded this state

        if is_goal(state):
            # reconstruct path
            path = []
            s = state
            while s is not None:
                path.append(s)
                s = parent[s]
            return list(reversed(path)), expansions

        for nxt in neighbors(state):
            new_cost = cost + 1  # unit step cost

            if nxt not in cost_so_far or new_cost < cost_so_far[nxt]:
                cost_so_far[nxt] = new_cost
                parent[nxt] = state
                heapq.heappush(frontier, (new_cost, nxt))

    return None, expansions

Modify BFS & DFS to Count Expansions

In [6]:
def bfs_with_expansion(start, is_goal, neighbors):
    frontier = deque([start])
    parent = {start: None}
    expansions = 0

    while frontier:
        s = frontier.popleft()
        expansions += 1

        if is_goal(s):
            path = []
            while s is not None:
                path.append(s)
                s = parent[s]
            return list(reversed(path)), expansions

        for n in neighbors(s):
            if n not in parent:
                parent[n] = s
                frontier.append(n)
    return None, expansions

In [4]:
def dfs_with_expansion(start, is_goal, neighbors, limit=10000):
    stack = [start]
    parent = {start: None}
    expansions = 0

    while stack and expansions < limit:
        s = stack.pop()
        expansions += 1

        if is_goal(s):
            path = []
            while s is not None:
                path.append(s)
                s = parent[s]
            return list(reversed(path)), expansions

        for n in neighbors(s):
            if n not in parent:
                parent[n] = s
                stack.append(n)

    return None, expansions

In [8]:
from collections import deque
import heapq

def mc_neighbors(state):
    """Return legal successor states for (M_left, C_left, boat_side)."""
    M, C, side = state                  # Unpack state; side: 0=left, 1=right
    moves = [(1,0),(2,0),(0,1),(0,2),(1,1)]  # Boat can carry these combinations
    result = []                          # Accumulate legal next states here
    for m, c in moves:                   # Try each boat payload
        if side == 0:                    # If boat is on left bank
            M2, C2, side2 = M - m, C - c, 1   # Move people from left to right
        else:                            # If boat is on right bank
            M2, C2, side2 = M + m, C + c, 0   # Move people from right to left
        if 0 <= M2 <= 3 and 0 <= C2 <= 3:      # Keep counts within [0,3]
            # Safety on left: missionaries not outnumbered (unless zero missionaries)
            safe_left = (M2 == 0 or M2 >= C2)
            # Safety on right (computed from complements 3-M2, 3-C2)
            safe_right = ((3 - M2) == 0) or ((3 - M2) >= (3 - C2))
            if safe_left and safe_right:       # Only accept safe states
                result.append((M2, C2, side2)) # Add the legal successor
    return result                              # Return list of legal neighbors

def ucs(start, is_goal, neighbors):
    """
    Uniform-Cost Search (UCS) using a priority queue (min-heap).
    Every action has cost = 1.
    Returns (path, expansions)
    """
    frontier = [(0, start)]      # (cost, state)
    parent = {start: None}       # for reconstructing path
    cost_so_far = {start: 0}     # g(n)
    expansions = 0               # to count nodes expanded

    while frontier:
        cost, state = heapq.heappop(frontier)
        expansions += 1          # we expanded this state

        if is_goal(state):
            # reconstruct path
            path = []
            s = state
            while s is not None:
                path.append(s)
                s = parent[s]
            return list(reversed(path)), expansions

        for nxt in neighbors(state):
            new_cost = cost + 1  # unit step cost

            if nxt not in cost_so_far or new_cost < cost_so_far[nxt]:
                cost_so_far[nxt] = new_cost
                parent[nxt] = state
                heapq.heappush(frontier, (new_cost, nxt))

    return None, expansions

def bfs_with_expansion(start, is_goal, neighbors):
    frontier = deque([start])
    parent = {start: None}
    expansions = 0

    while frontier:
        s = frontier.popleft()
        expansions += 1

        if is_goal(s):
            path = []
            while s is not None:
                path.append(s)
                s = parent[s]
            return list(reversed(path)), expansions

        for n in neighbors(s):
            if n not in parent:
                parent[n] = s
                frontier.append(n)
    return None, expansions

def dfs_with_expansion(start, is_goal, neighbors, limit=10000):
    stack = [start]
    parent = {start: None}
    expansions = 0

    while stack and expansions < limit:
        s = stack.pop()
        expansions += 1

        if is_goal(s):
            path = []
            while s is not None:
                path.append(s)
                s = parent[s]
            return list(reversed(path)), expansions

        for n in neighbors(s):
            if n not in parent:
                parent[n] = s
                stack.append(n)

    return None, expansions

Run All Three Searches on Missionaries & Cannibals

In [10]:
start = (3, 3, 0)
goal = (0, 0, 1)

# BFS
bfs_path, bfs_exp = bfs_with_expansion(start, lambda s: s == goal, mc_neighbors)

# DFS
dfs_path, dfs_exp = dfs_with_expansion(start, lambda s: s == goal, mc_neighbors)

# UCS
ucs_path, ucs_exp = ucs(start, lambda s: s == goal, mc_neighbors)

print(" RESULTS ")
print("BFS: path length =", len(bfs_path), "| expansions =", bfs_exp)
print("DFS: path length =", len(dfs_path), "| expansions =", dfs_exp)
print("UCS: path length =", len(ucs_path), "| expansions =", ucs_exp)

 RESULTS 
BFS: path length = 12 | expansions = 15
DFS: path length = 12 | expansions = 12
UCS: path length = 12 | expansions = 15
